# Custom Implementation Example: Extending the Prompt Optimization Framework

This notebook demonstrates how to extend the framework with custom implementations:

1. Creating a custom PerformanceAnalyzer with specific metrics
2. Implementing a custom AdaptiveLearningManager with domain-specific optimization strategies
3. Creating a specialized PromptLibrary with enhanced capabilities
4. Building a complete end-to-end workflow for your specific use case

In [ ]:
# Import core components
import os
import json
import time
import asyncio
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Any, List, Optional
from pprint import pprint

# Import our framework components
from ailf.cognition.prompt_library import PromptLibrary
from ailf.schemas.prompt_engineering import PromptTemplateV1, PromptLibraryConfig
from ailf.feedback.performance_analyzer import PerformanceAnalyzer
from ailf.feedback.adaptive_learning_manager import AdaptiveLearningManager

## 1. Creating a Custom PerformanceAnalyzer 

Let's create a specialized performance analyzer that focuses on specific metrics relevant to a customer service chatbot scenario.

In [ ]:
class CustomerServicePerformanceAnalyzer(PerformanceAnalyzer):
    """
    A specialized performance analyzer for customer service chatbot scenarios.
    It tracks additional metrics like:
    - Resolution rate (percentage of conversations that resolved the issue)
    - Customer satisfaction scores
    - Handoff rate (percentage of conversations handed to human agents)
    - Response time metrics
    """
    
    def __init__(self, interaction_data=None):
        """Initialize with optional interaction data."""
        self.interaction_data = interaction_data or []
        self.metrics_cache = {}  # Cache for computed metrics
        
    def add_interaction(self, interaction_data):
        """Add a new interaction to the dataset."""
        self.interaction_data.append(interaction_data)
        # Clear cache since data has changed
        self.metrics_cache = {}
    
    def analyze_prompt_success(self):
        """Analyze success metrics for each prompt template used in interactions."""
        if "prompt_analysis" in self.metrics_cache:
            return self.metrics_cache["prompt_analysis"]
            
        prompt_metrics = {}
        
        for interaction in self.interaction_data:
            template_id = interaction.get("template_id")
            if not template_id:
                continue
                
            # Ensure template exists in the metrics dict
            if template_id not in prompt_metrics:
                prompt_metrics[template_id] = {
                    "total_uses": 0,
                    "successful_outcomes": 0,
                    "error_count": 0,
                    "satisfaction_scores": [],
                    "response_times": [],
                    "resolution_count": 0,
                    "handoff_count": 0
                }
            
            # Update metrics
            prompt_metrics[template_id]["total_uses"] += 1
            
            if interaction.get("error"):
                prompt_metrics[template_id]["error_count"] += 1
                
            if interaction.get("successful", False):
                prompt_metrics[template_id]["successful_outcomes"] += 1
                
            if interaction.get("resolved", False):
                prompt_metrics[template_id]["resolution_count"] += 1
                
            if interaction.get("handoff", False):
                prompt_metrics[template_id]["handoff_count"] += 1
                
            if "satisfaction_score" in interaction:
                prompt_metrics[template_id]["satisfaction_scores"].append(interaction["satisfaction_score"])
                
            if "response_time" in interaction:
                prompt_metrics[template_id]["response_times"].append(interaction["response_time"])
        
        # Calculate derived metrics
        for template_id, metrics in prompt_metrics.items():
            total = metrics["total_uses"]
            if total > 0:
                # Calculate rates
                metrics["error_rate"] = metrics["error_count"] / total
                metrics["success_rate"] = metrics["successful_outcomes"] / total
                metrics["resolution_rate"] = metrics["resolution_count"] / total
                metrics["handoff_rate"] = metrics["handoff_count"] / total
                
                # Calculate averages
                if metrics["satisfaction_scores"]:
                    metrics["average_satisfaction"] = sum(metrics["satisfaction_scores"]) / len(metrics["satisfaction_scores"])
                else:
                    metrics["average_satisfaction"] = None
                    
                if metrics["response_times"]:
                    metrics["average_response_time"] = sum(metrics["response_times"]) / len(metrics["response_times"])
                else:
                    metrics["average_response_time"] = None
        
        # Cache the results
        self.metrics_cache["prompt_analysis"] = prompt_metrics
        return prompt_metrics
    
    def get_customer_satisfaction_metrics(self):
        """Calculate customer satisfaction metrics across all interactions."""
        satisfaction_scores = []
        template_satisfaction = {}
        
        for interaction in self.interaction_data:
            if "satisfaction_score" not in interaction:
                continue
                
            score = interaction["satisfaction_score"]
            satisfaction_scores.append(score)
            
            # Track by template
            template_id = interaction.get("template_id")
            if template_id:
                if template_id not in template_satisfaction:
                    template_satisfaction[template_id] = []
                template_satisfaction[template_id].append(score)
        
        # Calculate overall satisfaction
        overall_avg = sum(satisfaction_scores) / len(satisfaction_scores) if satisfaction_scores else None
        
        # Calculate per-template satisfaction
        template_avg = {}
        for template_id, scores in template_satisfaction.items():
            template_avg[template_id] = sum(scores) / len(scores) if scores else None
            
        return {
            "overall_satisfaction": overall_avg,
            "template_satisfaction": template_avg,
            "satisfaction_count": len(satisfaction_scores),
            "distribution": {
                "excellent (4-5)": sum(1 for s in satisfaction_scores if s >= 4),
                "good (3-4)": sum(1 for s in satisfaction_scores if 3 <= s < 4),
                "fair (2-3)": sum(1 for s in satisfaction_scores if 2 <= s < 3),
                "poor (1-2)": sum(1 for s in satisfaction_scores if 1 <= s < 2),
                "very poor (0-1)": sum(1 for s in satisfaction_scores if s < 1),
            }
        }

# Create some sample interaction data for a customer service chatbot scenario
interaction_data = [
    # Greeting template interactions
    *[{
        "template_id": "greeting", 
        "successful": True,
        "error": False,
        "resolved": False,
        "handoff": False,
        "satisfaction_score": 4.5 if i % 10 != 0 else 3.0,
        "response_time": 0.8 + (i % 5) * 0.1
    } for i in range(50)],
    
    # Order status template interactions
    *[{
        "template_id": "order_status",
        "successful": i % 4 != 0,
        "error": i % 4 == 0,
        "resolved": i % 3 == 0,
        "handoff": i % 5 == 0,
        "satisfaction_score": 3.0 + (i % 10) * 0.2,
        "response_time": 1.5 + (i % 5) * 0.3
    } for i in range(40)],
    
    # Return policy template interactions
    *[{
        "template_id": "return_policy",
        "successful": i % 2 == 0,  # 50% success rate
        "error": i % 3 == 0,  # 33% error rate
        "resolved": i % 2 == 0,  # 50% resolution rate  
        "handoff": i % 3 == 0,  # 33% handoff rate
        "satisfaction_score": 2.0 + (i % 5) * 0.5,  # Lower satisfaction
        "response_time": 2.0 + (i % 4) * 0.5  # Longer response times
    } for i in range(30)]
]

# Initialize our custom analyzer with sample data
customer_analyzer = CustomerServicePerformanceAnalyzer(interaction_data)

# Display the specialized metrics
prompt_analysis = customer_analyzer.analyze_prompt_success()
satisfaction_metrics = customer_analyzer.get_customer_satisfaction_metrics()

# Convert to DataFrame for visualization
df = pd.DataFrame({
    template_id: {
        "success_rate": metrics["success_rate"],
        "error_rate": metrics["error_rate"],
        "resolution_rate": metrics["resolution_rate"],
        "handoff_rate": metrics["handoff_rate"],
        "avg_satisfaction": metrics["average_satisfaction"],
        "avg_response_time": metrics["average_response_time"]
    }
    for template_id, metrics in prompt_analysis.items()
}).T

# Display the metrics
print("Customer Service Performance Metrics:")
display(df)

# Display satisfaction distribution
print("\nCustomer Satisfaction Distribution:")
for category, count in satisfaction_metrics["distribution"].items():
    print(f"- {category}: {count} interactions")

# Visualize key metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot success rates
df[["success_rate", "resolution_rate"]].plot(kind="bar", ax=axes[0, 0])
axes[0, 0].set_title("Success & Resolution Rates by Template")
axes[0, 0].set_ylim(0, 1)

# Plot error and handoff rates
df[["error_rate", "handoff_rate"]].plot(kind="bar", ax=axes[0, 1])
axes[0, 1].set_title("Error & Handoff Rates by Template")
axes[0, 1].set_ylim(0, 1)

# Plot average satisfaction
df["avg_satisfaction"].plot(kind="bar", ax=axes[1, 0])
axes[1, 0].set_title("Average Satisfaction by Template")
axes[1, 0].set_ylim(0, 5)  # Assuming 5-star scale

# Plot average response time
df["avg_response_time"].plot(kind="bar", ax=axes[1, 1])
axes[1, 1].set_title("Average Response Time by Template (seconds)")

plt.tight_layout()
plt.show()

## 2. Creating a Custom AdaptiveLearningManager

Now let's create a specialized AdaptiveLearningManager for customer service templates that has domain-specific optimization strategies.

In [ ]:
class CustomerServiceLearningManager(AdaptiveLearningManager):
    """
    A specialized AdaptiveLearningManager for customer service chatbots with:
    - Domain-specific optimization strategies
    - Customer satisfaction-focused improvements
    - Response time optimization
    - Handoff reduction strategies
    """
    
    def __init__(self, performance_analyzer, prompt_library=None, config=None, ai_engine=None):
        """Initialize with customer service specific configuration defaults."""
        # Set default config values for customer service scenarios
        default_config = {
            "error_rate_threshold": 0.2,  # Lower threshold for customer-facing services
            "feedback_optimization_threshold": 0.6,  # Higher bar for customer satisfaction
            "resolution_rate_threshold": 0.7,  # Expect 70%+ resolution rate
            "handoff_rate_threshold": 0.2,  # Keep handoffs below 20%
            "response_time_threshold": 2.0,  # Target response time below 2 seconds
            "auto_optimize_prompts": True
        }
        
        # Override defaults with provided config
        if config:
            default_config.update(config)
            
        # Call parent initializer with updated config
        super().__init__(
            performance_analyzer=performance_analyzer,
            prompt_library=prompt_library,
            config=default_config,
            ai_engine=ai_engine
        )
    
    async def _generate_rule_based_improvement(self, template, metrics, suggestion):
        """
        Enhanced rule-based improvement with customer service specific strategies.
        """
        prompt = template.user_prompt_template
        improvements_applied = []
        
        # Strategy 1: Improve clarity for high error rates
        if metrics.get("error_rate", 0) > self.config.get("error_rate_threshold", 0.2):
            original_prompt = prompt
            prompt = prompt.replace("provide", "provide step-by-step")
            prompt = prompt.replace("tell me", "explain clearly")
            if prompt != original_prompt:
                improvements_applied.append("Improved clarity for error reduction")
            
        # Strategy 2: Add empathy for low satisfaction scores
        if metrics.get("average_satisfaction", 5) < self.config.get("feedback_optimization_threshold", 4):
            if not any(word in prompt.lower() for word in ["sorry", "understand", "appreciate"]):
                if "?" in prompt:
                    prompt = prompt.replace("?", "? I understand this might be frustrating, ")
                else:
                    prompt += " I'm here to help you with this matter."
                improvements_applied.append("Added empathy for improved satisfaction")
        
        # Strategy 3: Add specificity for high handoff rates
        if metrics.get("handoff_rate", 0) > self.config.get("handoff_rate_threshold", 0.2):
            if not "specific" in prompt.lower():
                prompt = prompt.replace(".", ". Please provide specific details so I can assist you better.")
                improvements_applied.append("Added request for specific details to reduce handoffs")
        
        # Strategy 4: Streamline for slow response times
        if metrics.get("average_response_time", 0) > self.config.get("response_time_threshold", 2.0):
            # Find and remove verbose phrases
            verbose_phrases = ["if you don't mind", "would you be so kind", "if it's not too much trouble"]
            for phrase in verbose_phrases:
                prompt = prompt.replace(phrase, "")
            if not any(phrase in prompt for phrase in verbose_phrases):
                # Already concise, just note it
                improvements_applied.append("Maintained concise phrasing for response time")
            else:
                improvements_applied.append("Streamlined wording for faster response time")
                
        # If no improvements were made, return the original
        if not improvements_applied:
            return template.user_prompt_template
            
        # Return the improved prompt with optimization notes
        return prompt
    
    async def suggest_customer_service_improvements(self, underperforming_prompts):
        """
        Suggest customer service specific improvements based on performance data.
        """
        if not underperforming_prompts:
            return {}
            
        suggestions = {}
        for prompt_id, data in underperforming_prompts.items():
            if data.get("handoff_rate", 0) > self.config.get("handoff_rate_threshold", 0.2):
                suggestions[prompt_id] = {
                    "issue": "High handoff rate",
                    "suggestion": "Add more specific information gathering questions and provide more detailed responses."
                }
            elif data.get("average_satisfaction", 5) < self.config.get("feedback_optimization_threshold", 3.5):
                suggestions[prompt_id] = {
                    "issue": "Low customer satisfaction",
                    "suggestion": "Add empathetic language and acknowledgment of customer concerns."
                }
            elif data.get("error_rate", 0) > self.config.get("error_rate_threshold", 0.2):
                suggestions[prompt_id] = {
                    "issue": "High error rate",
                    "suggestion": "Simplify instructions and add step-by-step guidance."
                }
                
        return suggestions
    
    async def analyze_customer_service_metrics(self):
        """
        Generate specialized customer service insights from the performance data.
        """
        prompt_analysis = self.performance_analyzer.analyze_prompt_success()
        
        if not hasattr(self.performance_analyzer, "get_customer_satisfaction_metrics"):
            return {"error": "Performance analyzer doesn't support customer satisfaction metrics"}
            
        satisfaction_metrics = self.performance_analyzer.get_customer_satisfaction_metrics()
        
        # Find templates with concerning metrics
        concerning_templates = []
        for template_id, metrics in prompt_analysis.items():
            issues = []
            
            if metrics.get("handoff_rate", 0) > self.config.get("handoff_rate_threshold", 0.2):
                issues.append(f"High handoff rate: {metrics['handoff_rate']:.1%}")
                
            if metrics.get("average_satisfaction", 5) < self.config.get("feedback_optimization_threshold", 3.5):
                issues.append(f"Low satisfaction: {metrics['average_satisfaction']:.1f}/5.0")
                
            if metrics.get("resolution_rate", 1) < self.config.get("resolution_rate_threshold", 0.7):
                issues.append(f"Low resolution rate: {metrics['resolution_rate']:.1%}")
                
            if issues:
                concerning_templates.append({
                    "template_id": template_id,
                    "issues": issues,
                    "metrics": {k: metrics[k] for k in ["handoff_rate", "average_satisfaction", "resolution_rate", "error_rate", "success_rate"]}
                })
                
        return {
            "overall_satisfaction": satisfaction_metrics.get("overall_satisfaction"),
            "concerning_templates": concerning_templates,
            "optimization_opportunities": len(concerning_templates)
        }

# Initialize our specialized learning manager
customer_manager = CustomerServiceLearningManager(
    performance_analyzer=customer_analyzer,
    config={
        "handoff_rate_threshold": 0.25,  # Allow up to 25% handoffs
        "response_time_threshold": 1.5    # Target 1.5 second response time
    }
)

# Test the specialized analyzers and optimizers
service_insights = await customer_manager.analyze_customer_service_metrics()

# Display the customer service specific insights
print("Customer Service Insights:")
print(f"- Overall satisfaction: {service_insights['overall_satisfaction']:.2f}/5.0")
print(f"- Templates needing optimization: {len(service_insights['concerning_templates'])}")

for i, template in enumerate(service_insights['concerning_templates']):
    print(f"\n{i+1}. Template: {template['template_id']}")
    print(f"   Issues: {', '.join(template['issues'])}")
    metrics = template['metrics']
    for metric, value in metrics.items():
        if isinstance(value, float):
            print(f"   {metric}: {value:.2f}")

## 3. Creating a Specialized PromptLibrary 

Now let's create a PromptLibrary extension for customer service prompts with domain-specific features.

In [ ]:
# Create a temporary directory for our prompt templates
import tempfile
cs_prompt_library_path = tempfile.mkdtemp()
print(f"Created temporary directory for templates: {cs_prompt_library_path}")

class CustomerServicePromptLibrary(PromptLibrary):
    """
    Enhanced PromptLibrary with customer service specific features:
    - Special template categories for different service scenarios
    - Template recommendations based on customer context
    - A/B testing capabilities for measuring customer satisfaction impact
    - Enhanced versioning with roll-back capabilities
    """
    
    def __init__(self, config):
        """Initialize the customer service prompt library."""
        super().__init__(config)
        self._categories = {}  # Map of category -> template_ids
        self._categorize_templates()
    
    def _categorize_templates(self):
        """Categorize loaded templates based on their tags."""
        self._categories = {}
        for template_id, template in self._templates.items():
            for tag in template.tags:
                if tag not in self._categories:
                    self._categories[tag] = []
                self._categories[tag].append(template_id)
    
    def get_templates_by_category(self, category):
        """Get all templates in a specific category."""
        template_ids = self._categories.get(category, [])
        return [self.get_template(tid) for tid in template_ids]
    
    def list_categories(self):
        """List all available categories."""
        return list(self._categories.keys())
    
    def recommend_template(self, customer_context):
        """
        Recommend a template based on customer context.
        
        :param customer_context: Dict with details like issue_type, 
                                customer_history, sentiment, etc.
        :return: Best matching template or None
        """
        issue_type = customer_context.get("issue_type")
        if issue_type and issue_type in self._categories:
            # Find templates matching the issue type
            candidates = self._categories[issue_type]
            if candidates:
                # Simple recommendation - just return the first match
                # In real implementation, this would use more sophisticated matching
                return self.get_template(candidates[0])
        
        # Fallback to default if available
        if self.config.default_prompt_id:
            return self.get_template(self.config.default_prompt_id)
        
        return None
    
    def rollback_template(self, template_id, to_version):
        """
        Roll back a template to a previous version.
        
        :param template_id: The ID of the template
        :param to_version: The version to roll back to
        :return: The rolled-back template or None if not possible
        """
        # Find the target version
        target = None
        if template_id in self._version_history:
            for version in self._version_history[template_id]:
                if version.version == to_version:
                    target = version
                    break
        
        # If not in history, check if it's the current version
        if not target and template_id in self._templates:
            current = self._templates[template_id]
            if current.version == to_version:
                target = current
        
        if not target:
            return None
        
        # Create a new version based on the target
        rollback = copy.deepcopy(target)
        
        # Increment version and update metadata
        current_version = self._templates[template_id].version if template_id in self._templates else 0
        rollback.version = current_version + 1
        rollback.updated_at = time.time()
        rollback.updated_by_component = "rollback"
        rollback.version_notes = f"Rollback to version {to_version}"
        
        # Add the rollback version
        self.add_template(rollback, overwrite=True)
        
        return rollback

# Create a specialized config for customer service prompts
cs_config = PromptLibraryConfig(
    library_path=cs_prompt_library_path,
    default_prompt_id="greeting",
    auto_save=True
)

# Define sample customer service templates
cs_templates = [
    {
        "template_id": "greeting",
        "version": 1,
        "description": "Customer greeting template",
        "system_prompt": "You are a helpful customer service representative.",
        "user_prompt_template": "Welcome to our customer service. How can I assist you today?",
        "placeholders": [],
        "tags": ["greeting", "general"],
        "created_at": time.time()
    },
    {
        "template_id": "order_status",
        "version": 1,
        "description": "Template for checking order status",
        "system_prompt": "You are a helpful order management assistant.",
        "user_prompt_template": "I'll help you check the status of your order. Can you please provide your order number for {order_type}?",
        "placeholders": ["order_type"],
        "tags": ["orders", "status"],
        "created_at": time.time()
    },
    {
        "template_id": "return_policy",
        "version": 1,
        "description": "Template for explaining return policies",
        "system_prompt": "You are a helpful returns department assistant.",
        "user_prompt_template": "Here's our return policy for {product_category} items: Returns are accepted within {days} days of purchase.",
        "placeholders": ["product_category", "days"],
        "tags": ["returns", "policy"],
        "created_at": time.time()
    }
]

# Save templates to JSON files
for template_data in cs_templates:
    filename = f"{template_data['template_id']}_v{template_data['version']}.json"
    filepath = os.path.join(cs_prompt_library_path, filename)
    with open(filepath, 'w') as f:
        json.dump(template_data, f, indent=2)

# Initialize the specialized prompt library
cs_prompt_library = CustomerServicePromptLibrary(cs_config)

# Test the specialized features
print("Available categories:")
categories = cs_prompt_library.list_categories()
for category in categories:
    print(f"- {category}: {len(cs_prompt_library.get_templates_by_category(category))} templates")

# Test template recommendation
customer_context = {"issue_type": "returns", "sentiment": "frustrated"}
recommended = cs_prompt_library.recommend_template(customer_context)
if recommended:
    print(f"\nRecommended template for returns issue: {recommended.template_id} (v{recommended.version})")
    print(f"- Description: {recommended.description}")
    print(f"- Template: {recommended.user_prompt_template}")

## 4. Creating a Complete End-to-End Optimization Workflow

Now let's put it all together into a complete end-to-end workflow using our custom components.

In [ ]:
async def run_customer_service_optimization_workflow():
    """
    Demonstrate a complete end-to-end workflow for customer service prompt optimization.
    """
    print("Starting customer service optimization workflow...")
    
    # Step 1: Setup and initialization (already done above)
    library = cs_prompt_library
    analyzer = customer_analyzer
    manager = customer_manager
    
    # Connect the manager to the library
    manager.prompt_library = library
    
    # Step 2: Record initial state of templates
    print("\nInitial template states:")
    initial_templates = {}
    for template_id in library.list_template_ids():
        template = library.get_template(template_id)
        initial_templates[template_id] = template
        print(f"- {template_id} (v{template.version}): {template.user_prompt_template}")
    
    # Step 3: Analyze customer service metrics
    print("\nAnalyzing customer service metrics...")
    cs_metrics = await manager.analyze_customer_service_metrics()
    
    # Step 4: Identify underperforming templates
    underperforming = manager.identify_underperforming_prompts({
        "success_rate_threshold": 0.7,
        "min_sample_size": 10
    })
    
    print(f"\nIdentified {len(underperforming)} underperforming templates:")
    for template_id, data in underperforming.items():
        print(f"- {template_id}: Success rate {data.get('success_rate', 0):.2f}, Error rate {data.get('error_rate', 0):.2f}")
    
    # Step 5: Generate improvement suggestions
    print("\nGenerating customer service specific suggestions...")
    suggestions = await manager.suggest_customer_service_improvements(underperforming)
    
    print("Improvement suggestions:")
    for template_id, suggestion in suggestions.items():
        print(f"- {template_id}: {suggestion['issue']}")
        print(f"  Suggestion: {suggestion['suggestion']}")
    
    # Step 6: Run the optimization cycle with auto-apply
    print("\nRunning optimization cycle with auto-apply=True...")
    cycle_results = await manager.run_learning_cycle(auto_optimize=True)
    
    # Step 7: Check results and report changes
    print("\nOptimization cycle completed.")
    optimized_count = len(cycle_results.get('optimized_prompts', []))
    print(f"Optimized {optimized_count} templates")
    
    # Step 8: Show before/after comparison
    print("\nTemplate changes:")
    for template_id in library.list_template_ids():
        template = library.get_template(template_id)
        initial = initial_templates.get(template_id)
        
        if template.version > initial.version:
            print(f"\n{template_id} (v{initial.version} → v{template.version}):")
            print(f"BEFORE: {initial.user_prompt_template}")
            print(f"AFTER:  {template.user_prompt_template}")
            
            if hasattr(template, 'version_notes') and template.version_notes:
                print(f"NOTES:  {template.version_notes}")
    
    # Step 9: Create visualization of improvements
    if optimized_count > 0:
        # Collect metrics for visualization
        print("\nVisualizing changes in key metrics:")
        
        # This would normally compare before/after metrics
        # For this example, we'll just show the current metrics
        prompt_analysis = analyzer.analyze_prompt_success()
        
        # Extract key metrics for optimized templates
        metrics_df = pd.DataFrame({
            template_id: {
                "Handoff Rate": metrics.get("handoff_rate", 0),
                "Resolution Rate": metrics.get("resolution_rate", 0),
                "Satisfaction": metrics.get("average_satisfaction", 0),
                "Error Rate": metrics.get("error_rate", 0)
            }
            for template_id, metrics in prompt_analysis.items()
            if template_id in [t["template_id"] for t in cs_metrics.get("concerning_templates", [])]
        }).T
        
        # Display metrics
        if not metrics_df.empty:
            display(metrics_df)
            
            # Create visualization
            fig, ax = plt.subplots(figsize=(10, 6))
            metrics_df.plot(kind="bar", ax=ax)
            ax.set_title("Key Metrics for Optimized Templates")
            ax.set_ylim(0, 5)  # Assuming 0-5 scale for satisfaction, 0-1 for rates
            
            # Add threshold lines
            ax.axhline(y=manager.config["handoff_rate_threshold"], color='r', linestyle='--', 
                      label=f"Handoff Threshold ({manager.config['handoff_rate_threshold']})")
            ax.axhline(y=manager.config["resolution_rate_threshold"], color='g', linestyle='--',
                      label=f"Resolution Threshold ({manager.config['resolution_rate_threshold']})")
            
            ax.legend()
            plt.tight_layout()
            plt.show()
    
    return {
        "templates_analyzed": len(library.list_template_ids()),
        "templates_optimized": optimized_count,
        "optimization_cycle_id": cycle_results.get("cycle_id")
    }

# Run the complete workflow
workflow_results = await run_customer_service_optimization_workflow()
print(f"\nWorkflow completed: {workflow_results['templates_optimized']}/{workflow_results['templates_analyzed']} templates optimized.")

## Summary: Building Your Own Specialized Workflow

In this notebook, we've demonstrated how to extend the base prompt optimization framework with custom implementations for a specific domain (customer service). The key components we've built include:

1. **Specialized PerformanceAnalyzer**: Added domain-specific metrics like resolution rate, handoff rate, and customer satisfaction.

2. **Enhanced AdaptiveLearningManager**: Implemented customer service-specific optimization strategies focused on empathy, clarity, and response times.

3. **Extended PromptLibrary**: Added features like categorization, template recommendations, and rollback capabilities.

4. **Complete End-to-End Workflow**: Tied everything together into a cohesive process for analyzing and optimizing customer service prompts.

This pattern can be applied to any domain by:
- Identifying the domain-specific metrics that matter
- Creating specialized optimization strategies for those metrics
- Building domain-appropriate extensions to the core components
- Creating a workflow that ties them all together

The modular design of the framework makes it flexible enough to adapt to many different use cases while maintaining a consistent core functionality.